### From a File


In [1]:
from main import create_taxonomy, add_categories

filename = "cais_data.csv"
brand_terms = "CAIS"

taxonomy, df, samples = create_taxonomy(filename,
                                        text_column = "keyword",
                                        search_volume_column = "search_volume",
                                        taxonomy_model = "openai", # "palm" or "openai"
                                        days = 30,
                                        S=100,
                                        ngram_range = (1, 6),
                                        min_df = 2,
                                        brand_terms = None)

df = add_categories(taxonomy, df, brand) 

df.to_csv("cais_taxonomy.csv", index=False)

df.head()

,query,search_volume,original_query,taxonomy_category,similiary_score,taxonomy
0,1031 alternatives,50,1031 alternatives,Other Alternative,-0.521630,Other > Alternative
1,alternatives to masterclass,40,alternatives to masterclass,Other Alternative,-1.842644,Other > Alternative
2,apollo aligned alternatives,260,apollo aligned alternatives,Other Apollo Aligned Alternatives,5.211205,Other > Apollo Aligned Alternatives
3,glass alternative,260,glass alternative,Other Alternative,-0.090057,Other > Alternative
4,glass alternatives,260,glass alternatives,Other Alternative,-0.418348,Other > Alternative


### From a GSC Account

In [ ]:
from main import create_taxonomy

brand = "Green Group"

taxonomy, df, samples = create_taxonomy("sc-domain:greengroupcompanies.com",
                                        text_column = None,
                                        search_volume_column = None,
                                        platform = "openai", # "palm" or "openai"
                                        days = 30,
                                        S=100,
                                        ngram_range = (1, 6),
                                        min_df = 2,
                                        brand = None,
                                        limit_queries = 5)

df = add_categories(taxonomy, df, brand) 

df.to_csv("greengroupcompanies_taxonomy.csv", index=False)

df.head()

### Add clustered taxonomy

In [ ]:
from lib.clustering import ClusterTopics

queries = df['original_query'].tolist()

model = ClusterTopics(
        min_cluster_size =  3,
        min_samples = None,
        reduction_dims = 5,
        cluster_model = "agglomerative",
        cluster_categories = taxonomy,
        use_elbow = True,
        keep_outliers = False,
        n_jobs = 3,
    )


labels, text_labels = model.fit(queries)
label_lookup = {query: label for query, label in zip(queries, text_labels)}
df['clustered_taxonomy'] = df['original_query'].map(label_lookup)

df.to_csv("cais_taxonomy_clustered.csv", index=False)

df.head()